In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [5]:
df.shape

(57650, 4)

In [6]:
df=df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [7]:
#df=df.sample(5000)

In [8]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

In [9]:
df.tail(5)

,artist,song,text
4995,Andrea Bocelli,Malia,cosa c'era ne 'l fior che m'hai dato? \r fors...
4996,Donna Summer,Someday,someday \r when we are wiser \r when the wor...
4997,Judas Priest,Worth Fighting For,you said i could never find \r a way to ease ...
4998,Alabama,Christmas Shoes,it was almost christmas time \r there i stood...
4999,Counting Crows,A Murder Of One,blue morning blue morning \r wrapped in stran...


In [10]:
import nltk
from nltk.stem.porter import PorterStemmer

In [11]:
stemmer=PorterStemmer()

In [12]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w) for w in token]
    return " ".join(a)

In [13]:
token("you are beautiful")

'you are beauti'

In [14]:
df['text'].apply(lambda x:token(x))

0       oh she may be weari them young girl they do ge...
1       one shot oh lord you know , it 's gon na be a ...
2       fall farther with a flame hand i knew the ques...
3       trap out the hous with the board on the window...
4       now there 's newsprint all over your face well...
                              ...                        
4995    cosa c'era ne ' l fior che m'hai dato ? fors u...
4996    someday when we are wiser when the world 's ol...
4997    you said i could never find a way to eas your ...
4998    it wa almost christma time there i stood in an...
4999    blue morn blue morn wrap in strand of fist and...
Name: text, Length: 5000, dtype: object

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [17]:
metrix=tfid.fit_transform(df['text'])

In [18]:
similiar=cosine_similarity(metrix)

In [19]:
similiar[0]

array([1.        , 0.01239781, 0.15598231, ..., 0.01108372, 0.02579491,
       0.01405772])

In [21]:
df[df['song']=='Someday'].index[0]

4996

In [22]:
def reccommeder(song_name):
    idx=df[df['song']==song_name].index[0]
    distance=sorted(list(enumerate(similiar[idx])),reverse=True,key=lambda x:x[1])
    song=[]
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [24]:
reccommeder('Someday')

['Some Day', 'Someday Never Comes', 'I Still Believe', 'Rule The World']

In [25]:
import pickle

In [26]:
pickle.dump(similiar,open("similarity.pkl",'wb'))
pickle.dump(df,open("df.pkl",'wb'))